In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
data = pd.read_csv(r'C:\Users\ADMIN\sdsdatathon\data\champions_group_data.csv')

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_30460\2825282621.py:1: DtypeWarning: Columns (26,49) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(r'C:\Users\ADMIN\sdsdatathon\data\champions_group_data.csv')


In [10]:
data.shape

(8559, 72)

In [13]:
data['State Or Province Abbreviation'].isnull().sum()

np.int64(735)

In [14]:
print('Hello World')

Hello World


In [7]:
print("jibi")

jibi


In [8]:
print("Munhung is old")

Munhung is old
